In [1]:
import yfinance as yf
import pandas as pd

tickers = [
    "EWRK.ST","NOVO-B.CO","FOUR","BTS-B.ST","NWG.ST","EVO.ST","BETS-B.ST",
    "VOLV-B.ST","ERIC-B.ST","SAND.ST","ATCO-A.ST","HM-B.ST","INVE-B.ST",
    "SWED-A.ST","SHB-A.ST","SKF-B.ST","ASSA-B.ST","SEB-A.ST","TEL2-B.ST","SCA-B.ST",
    "INDU-C.ST","LATO-B.ST","BALD-B.ST","CAST.ST","HEXP.ST","KINV-B.ST",
    "ALFA.ST","LUN.ST","NIBE-B.ST","SSAB-A.ST"
]

rows = []
for t in tickers:
    try:
        stock = yf.Ticker(t)
        info = stock.info
        hist = stock.history(period="1d")

        rows.append({
            "Ticker": t,
            "Price": hist["Close"].iloc[-1] if not hist.empty else None,
            "MarketCap": info.get("marketCap"),
            "PE": info.get("trailingPE"),
            "PB": info.get("priceToBook"),
            "EV/EBITDA": info.get("enterpriseToEbitda"),
            "ROE": info.get("returnOnEquity"),
            "Debt/Equity": info.get("debtToEquity"),
            "Revenue": info.get("totalRevenue"),
            "NetIncome": info.get("netIncomeToCommon"),
            "OperatingMargin": info.get("operatingMargins"),
            "GrossMargins": info.get("grossMargins"),
            "FreeCashflow": info.get("freeCashflow"),
            "Beta": info.get("beta"),
            "DividendYield": info.get("dividendYield")
        })
    except Exception as e:
        print(f"Fel med {t}: {e}")

# Skapa DataFrame
df = pd.DataFrame(rows)

# Ta bort .ST från tickers
df["Ticker"] = df["Ticker"].str.replace(".ST", "", regex=False)

# Kolumner som ska ha tusentalsavgränsning och inga decimaler
numeric_cols_int = ["Price", "MarketCap", "Revenue", "NetIncome", "FreeCashflow"]

# Kolumner som ska avrundas till heltal
numeric_cols_round = ["PE", "PB", "EV/EBITDA", "ROE", "Debt/Equity", "Beta", "DividendYield"]

# Kolumner som ska ha 2 decimaler
numeric_cols_2dec = ["OperatingMargin", "GrossMargins"]

# Ersätt NaN med 0
df = df.fillna(0)

# Formatera stora tal (inga decimaler, tusentalsavgränsning)
for col in numeric_cols_int:
    df[col] = df[col].astype(float).round(0).astype(int).apply(lambda x: f"{x:,}".replace(",", " "))

# Runda nyckeltal till heltal
for col in numeric_cols_round:
    df[col] = df[col].astype(float).round(0).astype(int)

# Runda marginaler till två decimaler
for col in numeric_cols_2dec:
    df[col] = df[col].astype(float).round(2)

# Spara ren fil (skriv över den befintliga)
df.to_csv("fundamental_data.csv", index=False, encoding="utf-8-sig")

print("✅ Data tvättad och sparad till fundamental_data.csv")
print(df.head())


$NWG.ST: possibly delisted; no price data found  (period=1d) (Yahoo error = "No data found, symbol may be delisted")
HTTP Error 404: {"quoteSummary":{"result":null,"error":{"code":"Not Found","description":"Quote not found for symbol: HEXP.ST"}}}
$HEXP.ST: possibly delisted; no price data found  (period=1d) (Yahoo error = "No data found, symbol may be delisted")
$LUN.ST: possibly delisted; no price data found  (period=1d) (Yahoo error = "No data found, symbol may be delisted")


✅ Data tvättad och sparad till fundamental_data.csv
      Ticker Price          MarketCap  PE  PB  EV/EBITDA  ROE  Debt/Equity  \
0       EWRK   107      1 849 738 496  16   8         12    1          116   
1  NOVO-B.CO   364  1 619 630 227 456  15  10         11    1           59   
2       FOUR    76      6 677 433 856  28   8         12    0          187   
3      BTS-B   189      3 665 999 104  11   2         11    0           34   
4        NWG     0                  0   0   0          0    0            0   

           Revenue        NetIncome  OperatingMargin  GrossMargins  \
0   14 499 482 624      117 594 000             0.01          0.02   
1  311 937 990 656  111 067 996 160             0.44          0.84   
2    3 610 700 032      207 700 000             0.09          0.31   
3    2 821 441 024      337 822 016             0.09          0.43   
4                0                0             0.00          0.00   

     FreeCashflow  Beta  DividendYield  
0     212 415 120